In [ ]:
import marimo as mo

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
diabete = load_diabetes()
X = diabete.data
y = diabete.target

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

In [ ]:
Xtr, Xte, ytr, yte = train_test_split(X, y)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso 
from sklearn.neural_network import MLPRegressor

In [ ]:
lr = LinearRegression()
lrscore = cross_val_score(lr, Xtr, ytr)
lrscore

## Ajustement Lasso

**EXERCICE** Regarder l'impact de l'ajustement du paramètre $\alpha$ dans le modèle Lasso sur le score de cross validation.

In [ ]:
la = Lasso()

In [ ]:
gcv = GridSearchCV(
    Lasso(), 
    param_grid={
        "alpha": [10**p for p in range(-5, 3)]
}
)

In [ ]:
gcv.fit(Xtr, ytr)
gcv.best_params_, gcv.best_score_

In [ ]:
gcv.cv_results_

**REMARQUE** Le meilleur hyperparamètre pour le Lasso semble être $\alpha=0.01$

In [ ]:
cross_val_score(LinearRegression(), Xtr, ytr)

In [ ]:
cross_val_score(Lasso(alpha=0.01), Xtr, ytr)

**EXERCICE** Regarder comment fonctionne `RandomizedSearchCV`.

In [ ]:
import scipy.stats as stats

In [ ]:
normale = stats.norm()
normale

In [ ]:
for _ in range(10):
    print(normale.rvs())

In [ ]:
rcv = RandomizedSearchCV(Lasso(), dict(alpha=stats.expon()), n_iter=50)

In [ ]:
rcv.fit(Xtr, ytr)
rcv.best_params_, rcv.best_score_

## Ajustement du réseau de neurones

In [ ]:
nng = GridSearchCV(
    MLPRegressor(), 
    param_grid=dict(
       hidden_layer_sizes=[
           (3,), (9,), (15,)
       ],
        max_iter=[100000],
    )
)

In [ ]:
nng.fit(Xtr, ytr)
nng.best_params_, nng.best_score_

**EXERCICE** déterminer le nombre de paramètres pour chacun des choix d'hyperparamètres.

Correction au tableau: $12 n +1* paramètres pour une couche cachée de $n$ neurones.

## Preprocessing

[la documentation de MLP](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use)
suggère que la normalisation des données est critique pour les réseaux de neurones.

On va assembler un pipeline normalisation suivant du réseau de neurones.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
assemblage = Pipeline(
    steps=[
        ("normalisation", StandardScaler()),
        ("prediction", MLPRegressor())
    ]
)

In [ ]:
assemblage.get_params()

In [ ]:
gca = GridSearchCV( 
    assemblage, 
    param_grid=dict(
        prediction__hidden_layer_sizes=[
            (5,), (10,), (15,),
        ], 
        prediction__max_iter=[100000],
    )
)

In [ ]:
gca.fit(Xtr, ytr)
gca.best_params_, gca.best_score_

In [ ]:
gcm = GridSearchCV(
    Pipeline(
        steps=[
            ("normalisation", MinMaxScaler()),
            ("prediction", MLPRegressor()),
        ],
    ),
    param_grid=dict(
        prediction__hidden_layer_sizes=[
            (5,), (10,), (15,),
        ], 
        prediction__max_iter=[100000],
    )
)

In [ ]:
gcm.fit(Xtr, ytr)
gcm.best_params_, gca.best_score_

**EXERCICE** Faite la même chose pour les modèles `KNeighborsRegressor` et `GaussianProcessRegressor`.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
gknr = GridSearchCV(
    KNeighborsRegressor(),
    param_grid=dict(
        n_neighbors=range(2, 15),
        weights=["uniform", "distance"],
    )
)

In [ ]:
gknr.fit(Xtr, ytr)
gknr.best_params_, gknr.best_score_